# Exploration des données — Electio-Analytics

Notebook d'exploration de la base SQLite `electio_herault.db`.  
Données filtrées pour le département de l'Hérault (34), 341 communes.

**Prérequis** : avoir exécuté le pipeline ETL (`python main.py etl`) pour générer la base.

In [16]:
import sqlite3
import pandas as pd

DB_PATH = "data/output/electio_herault.db"
conn = sqlite3.connect(DB_PATH)
print(f"Connecté à {DB_PATH}")

Connecté à data/output/electio_herault.db


## 1. Vue d'ensemble de la base

12 tables, toutes liées par `codgeo` (code INSEE commune).

In [17]:
# Lister toutes les tables et leur nombre de lignes
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)

print(f"{'Table':<25s} {'Lignes':>8s}")
print("-" * 35)
for table in tables['name']:
    count = pd.read_sql(f"SELECT COUNT(*) as nb FROM {table}", conn)['nb'][0]
    print(f"  {table:<23s} {count:>8,}")

Table                       Lignes
-----------------------------------
  catnat                     4,130
  communes                     341
  comptes_communes           5,477
  csp                        3,087
  csp_diplome                2,744
  diplomes                     341
  elections                 19,596
  naissances_deces           5,797
  population                12,617
  revenus                      342
  risques                    3,184
  secteurs_activite          3,087


## 2. Communes (table de référence)

Table pivot du modèle : chaque commune a un `codgeo` unique (code INSEE 5 caractères).

In [18]:
communes = pd.read_sql("SELECT * FROM communes", conn)
print(f"Nombre de communes : {len(communes)}")
print(f"Colonnes : {list(communes.columns)}")
communes.sample(10)

Nombre de communes : 341
Colonnes : ['codgeo', 'nom', 'departement']


,codgeo,nom,departement
317,34320,Vailhauquès,34
296,34299,Sérignan,34
290,34293,La Salvetat-sur-Agout,34
209,34211,Le Poujol-sur-Orb,34
276,34279,Saint-Nazaire-de-Ladarez,34
142,34144,Lunas-les-Châteaux,34
339,34343,Viols-le-Fort,34
300,34303,Sorbs,34
99,34101,Florensac,34
91,34092,Cruzy,34


## 3. Élections municipales

Résultats par candidat, par commune, pour les municipales 2008, 2014, 2020.  
Chaque candidat est classé `Gauche` ou `Droite` (colonne `camp`).

In [19]:
elections = pd.read_sql("SELECT * FROM elections", conn)
print(f"Lignes : {len(elections):,}")
print(f"Années : {sorted(elections['annee'].unique())}")
print(f"Tours : {sorted(elections['tour'].unique())}")
print(f"\nRépartition par camp :")
print(elections['camp'].value_counts())
elections.head()

Lignes : 19,596
Années : [np.int64(2008), np.int64(2014), np.int64(2020)]
Tours : [np.int64(1), np.int64(2)]

Répartition par camp :
camp
Droite    13801
Gauche     5795
Name: count, dtype: int64


,codgeo,annee,tour,nom_candidat,prenom_candidat,nuance,voix,pct_voix_inscrits,pct_voix_exprimes,camp
0,34003,2020,2,COUSIN,Jean Louis,LRN,46,3.61,7.07,Droite
1,34003,2020,2,COUSIN,Jean Louis,LRN,34,3.36,8.56,Droite
2,34003,2020,2,COUSIN,Jean Louis,LRN,57,5.30,11.88,Droite
3,34003,2020,2,COUSIN,Jean Louis,LRN,33,3.36,7.86,Droite
4,34003,2020,2,COUSIN,Jean Louis,LRN,29,3.21,6.02,Droite


In [20]:
# Répartition des voix par année et camp (T1 uniquement)
t1 = elections[elections['tour'] == 1]
pivot = t1.groupby(['annee', 'camp'])['voix'].sum().unstack(fill_value=0)
pivot['total'] = pivot.sum(axis=1)
pivot['pct_gauche'] = (100 * pivot.get('Gauche', 0) / pivot['total']).round(1)
pivot

camp,Droite,Gauche,total,pct_gauche
annee,,,,
2008,185417,131460,316877,41.5
2014,765028,191801,956829,20.0
2020,631540,101998,733538,13.9


## 4. Population

Population municipale par commune et par année de recensement (1968 à 2022).

In [21]:
population = pd.read_sql("SELECT * FROM population", conn)
print(f"Lignes : {len(population):,}")
print(f"Années disponibles : {sorted(population['annee'].unique())}")
print(f"\nStatistiques sur la population :")
population['population'].describe().round(0)

Lignes : 12,617
Années disponibles : [np.int64(1876), np.int64(1881), np.int64(1886), np.int64(1891), np.int64(1896), np.int64(1901), np.int64(1906), np.int64(1911), np.int64(1921), np.int64(1926), np.int64(1931), np.int64(1936), np.int64(1954), np.int64(1962), np.int64(1968), np.int64(1975), np.int64(1982), np.int64(1990), np.int64(1999), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]

Statistiques sur la population :


count     12617.0
mean       2406.0
std       12483.0
min           5.0
25%         255.0
50%         622.0
75%        1719.0
max      310240.0
Name: population, dtype: float64

In [22]:
# Top 10 communes les plus peuplées (dernière année disponible)
derniere_annee = population['annee'].max()
top_pop = population[population['annee'] == derniere_annee].nlargest(10, 'population')
top_pop = top_pop.merge(communes[['codgeo', 'nom']], on='codgeo', how='left')
print(f"Top 10 communes en {derniere_annee} :")
top_pop[['nom', 'codgeo', 'population']]

Top 10 communes en 2023 :


,nom,codgeo,population
0,Montpellier,34172,310240
1,Béziers,34032,81545
2,Sète,34301,45337
3,Agde,34003,29939
4,Lunel,34145,26623
5,Castelnau-le-Lez,34057,26058
6,Frontignan,34108,24136
7,Lattes,34129,17351
8,Mauguio,34154,16522
9,Juvignac,34123,14055


## 5. Revenus

In [23]:
revenus = pd.read_sql("SELECT * FROM revenus", conn)
print(f"Lignes : {len(revenus)}")
print(f"Colonnes ({len(revenus.columns)}) :")
for c in revenus.columns:
    print(f"  - {c}")
revenus.head()

Lignes : 342
Colonnes (55) :
  - [disp]_nbre_de_menages_fiscaux
  - [disp]_nbre_de_personnes_dans_les_menages_fiscaux
  - [disp]_nbre_dunites_de_consommation_dans_les_menages_fiscaux
  - [disp]_1ᵉʳ_quartile_(€)
  - [disp]_mediane_(€)
  - [disp]_3ᵉ_quartile_(€)
  - [disp]_ecart_interquartile_(€)
  - [disp]_1ᵉʳ_decile_(€)
  - [disp]_2ᵉ_decile_(€)
  - [disp]3ᵉ_decile_(€)
  - [disp]_4ᵉ_decile_(€)
  - [disp]_6ᵉ_decile_(€)
  - [disp]_7ᵉ_decile_(€)
  - [disp]_8ᵉ_decile_(€)
  - [disp]_9ᵉ_decile_(€)
  - [disp]_rapport_interdecile_9ᵉ_decile/1ᵉʳ_decile
  - [disp]_s80/20
  - [disp]_iice_de_gini
  - [disp]_part_des_revenus_d’activite_(%)
  - [disp]_dont_part_des_salaires_et_traitements(%)
  - [disp]_dont_part_des_iemnites_de_chômage_(%)
  - [disp]_dont_part_des_revenus_des_activites_non_salariees_(%)
  - [disp]_part_des_pensions,_retraites_et_rentes_(%)
  - [disp]_part_des_revenus_du_patrimoine_et_autres_revenus_(%)
  - [disp]_part_de_lensemble_des_prestations_sociales_(%)
  - [disp]_dont_part_des_

,[disp]_nbre_de_menages_fiscaux,[disp]_nbre_de_personnes_dans_les_menages_fiscaux,[disp]_nbre_dunites_de_consommation_dans_les_menages_fiscaux,[disp]_1ᵉʳ_quartile_(€),[disp]_mediane_(€),[disp]_3ᵉ_quartile_(€),[disp]_ecart_interquartile_(€),[disp]_1ᵉʳ_decile_(€),[disp]_2ᵉ_decile_(€),[disp]3ᵉ_decile_(€),...,[dec]_rapport_interdecile_d9/d1,[dec]_s80/s20,[dec]_iice_de_gini,[dec]_part_des_revenus_d’activite_(%),[dec]_dont_part_des_salaires_et_traitements_(%),[dec]_dont_part_des_iemnites_de_chômage_(%),[dec]_dont_part_des_revenus_des_activites_non_salariees_(%),"[dec]_part_des_pensions,_retraites_et_rentes_(%)",[dec]_part_des_autres_revenus_(%),codgeo
0,771.0,1848.0,1244.1,NaN,21720.0,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34001
1,566.0,1348.0,902.8,NaN,21060.0,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34002
2,16947.0,31298.0,23442.4,14570.0,20410.0,27350.0,12780.0,10510.0,13370.0,15710.0,...,"8,0",10.7,0.389,46.5,36.9,4.2,5.4,44.0,9.5,34003
3,117.0,233.0,171.1,NaN,18830.0,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34004
4,124.0,281.0,191.8,NaN,23490.0,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34005


## 6. CSP (Catégories socio-professionnelles)

Actifs de 25-54 ans par catégorie (cadres, ouvriers, employés, etc.) et par année de recensement.

In [24]:
csp = pd.read_sql("SELECT * FROM csp", conn)
print(f"Lignes : {len(csp)}")
print(f"Années RP : {sorted(csp['annee'].unique())}")
print(f"\nColonnes ({len(csp.columns)}) :")
for c in csp.columns[:15]:
    print(f"  - {c}")
if len(csp.columns) > 15:
    print(f"  ... et {len(csp.columns) - 15} autres")

Lignes : 3087
Années RP : [np.int64(1968), np.int64(1975), np.int64(1982), np.int64(1990), np.int64(1999), np.int64(2006), np.int64(2011), np.int64(2016), np.int64(2022)]

Colonnes (110) :
  - agriculteurs
actifs_ayant_un_emploi
rp1968
  - agriculteurs
chômeurs
rp1968
  - artisans,_commerçants,_chefs_d'entreprise
actifs_ayant_un_emploi
rp1968
  - artisans,_commerçants,_chefs_d'entreprise
chômeurs
rp1968
  - cadres_et_professions_intellectuelles_supérieures
actifs_ayant_un_emploi
rp1968
  - cadres_et_professions_intellectuelles_supérieures
chômeurs
rp1968
  - professions_intermédiaires
actifs_ayant_un_emploi
rp1968
  - professions_intermédiaires
chômeurs
rp1968
  - employés
actifs_ayant_un_emploi
rp1968
  - employés
chômeurs
rp1968
  - ouvriers
actifs_ayant_un_emploi
rp1968
  - ouvriers
chômeurs
rp1968
  - codgeo
  - annee
  - agriculteurs
actifs_ayant_un_emploi
rp1975
  ... et 95 autres


## 7. Diplômes

Niveau de diplôme de la population de 15 ans et plus, par commune.  
Préfixes : `p11` (RP 2011), `p16` (RP 2016), `p22` (RP 2022).

In [25]:
diplomes = pd.read_sql("SELECT * FROM diplomes", conn)
print(f"Lignes : {len(diplomes)}")
print(f"\nColonnes ({len(diplomes.columns)}) :")
for c in diplomes.columns[:20]:
    print(f"  - {c}")
if len(diplomes.columns) > 20:
    print(f"  ... et {len(diplomes.columns) - 20} autres")

Lignes : 341

Colonnes (190) :
  - p22_pop0205
  - p22_pop0610
  - p22_pop1114
  - p22_pop1517
  - p22_pop1824
  - p22_pop2529
  - p22_pop30p
  - p22_scol0205
  - p22_scol0610
  - p22_scol1114
  - p22_scol1517
  - p22_scol1824
  - p22_scol2529
  - p22_scol30p
  - p22_h0205
  - p22_h0610
  - p22_h1114
  - p22_h1517
  - p22_h1824
  - p22_h2529
  ... et 170 autres


## 8. Comptes des communes

Données financières : dette, recettes, dépenses de fonctionnement et d'investissement.

In [26]:
comptes = pd.read_sql("SELECT * FROM comptes_communes", conn)
print(f"Lignes : {len(comptes)}")
print(f"Années : {sorted(comptes['annee'].unique())}")
print(f"Colonnes : {list(comptes.columns)}")
comptes.describe().round(0)

Lignes : 5477
Années : [np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2017), np.int64(2022)]
Colonnes : ['codgeo', 'annee', 'produits_fonctionnement', 'charges_fonctionnement', 'depenses_personnel', 'depenses_investissement', 'depenses_equipement', 'dette', 'dgf', 'capacite_autofinancement', 'impots_directs', 'impots_indirects']


,annee,produits_fonctionnement,charges_fonctionnement,depenses_personnel,depenses_investissement,depenses_equipement,dette,dgf,capacite_autofinancement,impots_directs,impots_indirects
count,5477.0,5477.0,5477.0,5477.0,5477.0,5477.0,5477.0,5477.0,5477.0,5477.0,5477.0
mean,2009.0,3496.0,3053.0,1542.0,1606.0,1082.0,2873.0,614.0,541.0,1618.0,306.0
std,6.0,17832.0,15552.0,8292.0,8320.0,5623.0,11968.0,3473.0,3202.0,9288.0,1460.0
min,2000.0,23.0,14.0,-8.0,0.0,0.0,0.0,4.0,-1404.0,-7.0,-21.0
25%,2004.0,206.0,173.0,62.0,92.0,73.0,85.0,49.0,30.0,60.0,19.0
50%,2008.0,583.0,479.0,206.0,283.0,226.0,370.0,126.0,85.0,220.0,44.0
75%,2014.0,1864.0,1590.0,717.0,917.0,725.0,1580.0,367.0,275.0,758.0,123.0
max,2022.0,357263.0,319633.0,191165.0,211991.0,151616.0,244424.0,75275.0,72488.0,229135.0,28182.0


## 9. Catastrophes naturelles (CatNat)

In [27]:
catnat = pd.read_sql("SELECT * FROM catnat", conn)
print(f"Lignes : {len(catnat)}")
print(f"Communes touchées : {catnat['codgeo'].nunique()}")
print(f"Colonnes : {list(catnat.columns)}")

# Top 10 communes les plus touchées
top_cat = catnat.groupby('codgeo').size().sort_values(ascending=False).head(10)
noms = communes.set_index('codgeo')['nom']
top_cat.index = top_cat.index.map(lambda x: f"{noms.get(x, x)} ({x})")
print(f"\nTop 10 communes les plus touchées :")
print(top_cat.to_string())

Lignes : 4130
Communes touchées : 342
Colonnes : ['codgeo', 'risque', 'date_debut', 'date_fin', 'date_arrete']

Top 10 communes les plus touchées :
codgeo
Montpellier (34172)                 37
Portiragnes (34209)                 37
Montarnaud (34163)                  36
Baillargues (34022)                 34
Marseillan (34150)                  34
Béziers (34032)                     32
Villeneuve-lès-Maguelone (34337)    32
Clapiers (34077)                    30
Saint-Gély-du-Fesc (34255)          30
Saint-Jean-de-Védas (34270)         29


## 10. Qualité des données

Vérification des valeurs manquantes et de la couverture par table.

In [28]:
tables_list = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)

for table in tables_list['name']:
    df = pd.read_sql(f"SELECT * FROM {table}", conn)
    nulls = df.isnull().sum()
    total_nulls = nulls.sum()
    if total_nulls > 0:
        print(f"\n{table} ({len(df)} lignes) — {total_nulls} valeurs manquantes :")
        for col, n in nulls[nulls > 0].items():
            print(f"  {col}: {n} ({100*n/len(df):.1f}%)")
    else:
        print(f"{table} ({len(df)} lignes) — OK")

catnat (4130 lignes) — OK
communes (341 lignes) — OK
comptes_communes (5477 lignes) — OK

csp (3087 lignes) — 296364 valeurs manquantes :
  agriculteurs
actifs_ayant_un_emploi
rp1968: 2744 (88.9%)
  agriculteurs
chômeurs
rp1968: 2744 (88.9%)
  artisans,_commerçants,_chefs_d'entreprise
actifs_ayant_un_emploi
rp1968: 2744 (88.9%)
  artisans,_commerçants,_chefs_d'entreprise
chômeurs
rp1968: 2744 (88.9%)
  cadres_et_professions_intellectuelles_supérieures
actifs_ayant_un_emploi
rp1968: 2744 (88.9%)
  cadres_et_professions_intellectuelles_supérieures
chômeurs
rp1968: 2744 (88.9%)
  professions_intermédiaires
actifs_ayant_un_emploi
rp1968: 2744 (88.9%)
  professions_intermédiaires
chômeurs
rp1968: 2744 (88.9%)
  employés
actifs_ayant_un_emploi
rp1968: 2744 (88.9%)
  employés
chômeurs
rp1968: 2744 (88.9%)
  ouvriers
actifs_ayant_un_emploi
rp1968: 2744 (88.9%)
  ouvriers
chômeurs
rp1968: 2744 (88.9%)
  agriculteurs
actifs_ayant_un_emploi
rp1975: 2744 (88.9%)
  agriculteurs
chômeurs
rp1975: 274

In [29]:
# Vérifier la couverture des jointures : combien de communes ont des données dans chaque table
codgeo_communes = set(communes['codgeo'])
tables_avec_codgeo = ['elections', 'population', 'revenus', 'csp', 'diplomes',
                       'comptes_communes', 'catnat', 'naissances_deces']

print(f"Communes de référence : {len(codgeo_communes)}")
print(f"\n{'Table':<25s} {'Communes':>10s} {'Couverture':>12s}")
print("-" * 50)
for table in tables_avec_codgeo:
    try:
        codgeos = set(pd.read_sql(f"SELECT DISTINCT codgeo FROM {table}", conn)['codgeo'])
        inter = codgeos & codgeo_communes
        pct = 100 * len(inter) / len(codgeo_communes)
        print(f"  {table:<23s} {len(inter):>8d}   {pct:>8.1f}%")
    except Exception as e:
        print(f"  {table:<23s} Erreur: {e}")

Communes de référence : 341

Table                       Communes   Couverture
--------------------------------------------------
  elections                    341      100.0%
  population                   341      100.0%
  revenus                      341      100.0%
  csp                          341      100.0%
  diplomes                     341      100.0%
  comptes_communes             341      100.0%
  catnat                       341      100.0%
  naissances_deces             341      100.0%


In [30]:
conn.close()
print("Connexion fermée.")

Connexion fermée.
